In [151]:
from aocd.models import Puzzle

puzzle = Puzzle(year=2024, day=13)

from parse import parse

pattern = """Button A: X+{:d}, Y+{:d}
Button B: X+{:d}, Y+{:d}
Prize: X={:d}, Y={:d}"""

def parses(data):
    return [parse(pattern, line).fixed 
            for line in data.strip().split('\n\n')]

data = parses(puzzle.input_data)

In [152]:
sample = parses("""Button A: X+94, Y+34
Button B: X+22, Y+67
Prize: X=8400, Y=5400

Button A: X+26, Y+66
Button B: X+67, Y+21
Prize: X=12748, Y=12176

Button A: X+17, Y+86
Button B: X+84, Y+37
Prize: X=7870, Y=6450

Button A: X+69, Y+23
Button B: X+27, Y+71
Prize: X=18641, Y=10279""")

In [ ]:
# a + xb/xa * b = px/xa
# a + yb/ya * b = py/ya

NameError: name 'px' is not defined

In [ ]:
# xa * a + xb * b = px
# ya * a + yb * b = py


a = (px-xb*b)/xa
a = px/xa - xb/xa * b

 - ya * xb/xa * b = py - ya * px/xa

b = (py - ya * px/xa) / (yb - ya * xb/xa)

In [170]:
A = np.array([[xa,xb],[ya,yb]])
b = np.array([px,py])
np.linalg.solve(A,b)

In [216]:
def presses(machine, delta=0):
    xa, ya, xb, yb, px, py = machine
    px, py = px+delta, py+delta
    b = (py - ya * px/xa) / (yb - ya * xb/xa)
    a = (px-xb*b) / xa
    a, b = round(a), round(b)
    if (xa * a + xb * b == px) and (ya * a + yb * b == py):
        return 3*a+b
    return 0

In [217]:
presses(sample[0])

280

In [218]:
def solve_a(data):
    return sum(presses(machine) for machine in data)

In [219]:
solve_a(data)

35255

In [220]:
def solve_b(data):
    return sum(presses(machine, 10000000000000) for machine in data)

In [221]:
solve_b(data)

87582154060429

In [2]:
from aocd.models import Puzzle

puzzle = Puzzle(year=2024, day=13)

# def parses(data):
#     return data.strip().split('\n')

import re

def extract_coords(line):
    return [int(i) for i in re.findall("-?\d+", line)]

def parse_group(group):
    a, b, prize = group.split('\n')
    return [extract_coords(x) for x in (a,b,prize)]

def parses(data):
    groups = data.strip().split('\n\n')
    return [parse_group(g) for g in groups]

data = parses(puzzle.input_data)

In [3]:
sample = parses("""Button A: X+94, Y+34
Button B: X+22, Y+67
Prize: X=8400, Y=5400

Button A: X+26, Y+66
Button B: X+67, Y+21
Prize: X=12748, Y=12176

Button A: X+17, Y+86
Button B: X+84, Y+37
Prize: X=7870, Y=6450

Button A: X+69, Y+23
Button B: X+27, Y+71
Prize: X=18641, Y=10279""")

In [4]:
sample

[[[94, 34], [22, 67], [8400, 5400]],
 [[26, 66], [67, 21], [12748, 12176]],
 [[17, 86], [84, 37], [7870, 6450]],
 [[69, 23], [27, 71], [18641, 10279]]]

In [5]:
import numpy as np

In [6]:
def mincost(a, b, prize):
    ax, ay = a
    bx, by = b
    px, py = prize
    mincost = float('inf')
    for i in range(min(px//ax,101)):
        rest = px - ax*i
        if rest % bx == 0:
            j = rest // bx
            if ax*i+bx*j == px and ay*i+by*j == py:
                mincost = min(mincost, 3*i+j)
#                 print(i,j)
        
    for j in range(min(px//bx,101)):
        rest = px - bx*j
        if rest % ax == 0:
            i = rest // ax
            if ax*i+bx*j == px and ay*i+by*j == py:
                mincost = min(mincost, 3*i+j)
#                 print(i,j)
    return mincost

In [7]:
def solve_a(data):
    total = 0
    for machine in data:
        cost = mincost(*machine)
        if cost != float('inf'):
            total += cost
    return total

In [8]:
solve_a(data)

35255

In [9]:
# solve_optimization()

In [10]:
from z3 import Int, Optimize, sat

def solve_optimization(a, b, p):
    xa, ya = a
    xb, yb = b
    px, py = p
    # Create Z3 solver
    solver = Optimize()
    
    # Define variables
    a = Int('a')
    b = Int('b')
    
    # Add constraints
    solver.add(a * xa + b * xb == px)  # First equation
    solver.add(a * ya + b * yb == py)  # Second equation
    
    # Define objective function to minimize
    objective = 3 * a + b
    
    # Add objective
    solver.minimize(objective)
    
    # Check if problem is satisfiable
    if solver.check() == sat:
        model = solver.model()
        return {
            'a': model[a].as_long(),
            'b': model[b].as_long(),
            'objective_value': 3 * model[a].as_long() + model[b].as_long()
        }
    else:
        return None


In [11]:
solve_optimization(*sample[1])

In [12]:
def solve_b(data):
    total = 0
    for machine in data:
        a, b, p = machine
        px, py = p
        px += 10000000000000
        py += 10000000000000
        cost = solve_optimization(a,b,(px,py))
        if cost is not None:
            total += cost['objective_value']
    return total

In [13]:
solve_b(data)

87582154060429

In [74]:
a,b,p = map(np.array, sample[3])

In [75]:
10000000000000 // a+b

array([144927536258, 434782608766])

In [76]:
a,b,p

(array([69, 23]), array([27, 71]), array([18641, 10279]))

In [77]:
for i in range(1,100):
    for j in range(1,100):
        x, y =a*i+b*j
        if x == y:
            print(i,j)

22 23
44 46
66 69
88 92


In [139]:
def mincost(a,b,p):
    ax, ay = a
    bx, by = b
    px, py = p
#     px += 10000000000000
#     py += 10000000000000
    reachable = {}
    
    for i in range(1,1000):
        for j in range(1,1000):
            pos = (ax*i+bx*j, ay*i+by*j)
            cost = 3*i+j
            if pos not in reachable or cost < reachable[pos]:
                reachable[pos] = cost
    equal = {pos[0]: cost for pos, cost in reachable.items() if pos[0]==pos[1]}
#     print(equal)
    mincost = float('inf')
    for (x, y), cost in reachable.items():
        Dx, Dy = px-x, py-y
        if Dx != Dy:
            continue
        for d, dcost in equal.items():
            if Dx % d == 0:
                n = Dx // d
                mincost = min(mincost, n * dcost + cost)
    return mincost

In [112]:
def increase(a,b,p):
    px, py = p
    px += 10000000000000
    py += 10000000000000
    return a,b,(px,py)

In [114]:
mincost(*increase(*sample[2]))

{6595: 210, 13190: 420, 19785: 630, 26380: 840}


inf

In [129]:
solve_optimization(*increase(*sample[1]))

{'a': 118679050709, 'b': 103199174542, 'objective_value': 459236326669}

In [130]:
mincost(*increase(*sample[1]))

459236326669

In [136]:
solve_b(data)

87582154060429

In [140]:
def solve_b2(data):
    total = 0
    for machine in data:
        cost = mincost(*increase(*machine))
        if cost != float('inf'):
            total += cost
    return total

In [141]:
solve_b2(data)

86236933949990

{'a': 80, 'b': 40, 'objective_value': 280}

In [23]:
from numba import njit

In [28]:
@njit
def mincost(a, b, prize):
    ax, ay = a
    bx, by = b
    px, py = prize
    mincost = float('inf')
    for i in range(px//ax):
        rest = px - ax*i
        if rest % bx == 0:
            j = rest // bx
            if ax*i+bx*j == px and ay*i+by*j == py:
                mincost = min(mincost, 3*i+j)
#                 print(i,j)
        
    for j in range(px//bx):
        rest = px - bx*j
        if rest % ax == 0:
            i = rest // ax
            if ax*i+bx*j == px and ay*i+by*j == py:
                mincost = min(mincost, 3*i+j)
#                 print(i,j)
    return mincost

In [ ]:
sample[0]

In [ ]:
mincost(*)

In [29]:
def solve_bad(data):
    total = 0
    for machine in data:
        a, b, p = machine
        px, py = p
        px += 10000000000000
        py += 10000000000000
        cost = mincost(a,b,(px,py))
        if cost != float('inf'):
            total += cost
    return total

In [ ]:
solve_bad(data)

In [31]:
mincost(*sample[2])

200

In [ ]:
def solve_a(data):
    pass

In [ ]:
solve_a(sample)

In [ ]:
solve_a(data)

In [ ]:
def solve_b(data):
    pass

In [ ]:
solve_b(sample)

In [ ]:
solve_b(data)